# Location Intelligence

https://www.nea.gov.sg/our-services/food-hygiene/licensing-permits/food-retail-industry/food-shop-licence

https://data.gov.sg/dataset/list-of-supermarket-licences?view_id=aa6bfba1-afbb-4932-b973-b1bc1c4bbf20&resource_id=3561a136-4ee4-4029-a5cd-ddf591cce643

In [86]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
import time
import re
import tqdm

import folium
import matplotlib as mpl

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [2]:
csv = "data\list-of-supermarket-licences.csv"
df = pd.read_csv(csv)
df.head()

,licence_num,licensee_name,building_name,block_house_num,level_num,unit_num,street_name,postal_code
0,S02185J000,COLD STORAGE SINGAPORE (1983) PTE LTD,na,982,1,1,BUANGKOK CRESCENT,530982
1,S97166P000,COLD STORAGE SINGAPORE (1983) PTE LTD,HOUGANG GREEN SHOPPING MALL,21,1,52,HOUGANG STREET 51,538719
2,CE15B61K000,U STARS SUPERMARKET PTE. LTD.,na,330,1,1,ANCHORVALE STREET,540330
3,S02029J000,NTUC Fairprice Co-operative Ltd,HOUGANG POINT,1,2,1,HOUGANG STREET 91,538692
4,CE04561V000,SHENG SIONG SUPERMARKET PTE LTD,na,19,1,42401,SERANGOON NORTH AVENUE 5,554913


In [3]:
len(df.licensee_name.unique())

146

In [16]:
df_unique_companies = df.groupby(df.licensee_name).size().sort_values(ascending=False)
top5 = list(df_unique_companies.head().keys())
top5

['NTUC Fairprice Co-operative Ltd',
 'COLD STORAGE SINGAPORE (1983) PTE LTD',
 'SHENG SIONG SUPERMARKET PTE LTD',
 'PRIME SUPERMARKET LIMITED?',
 'PRIME SUPERMARKET (1996) PTE LTD']

In [6]:
# plt.figure(figsize=(20,20))
# fig = sns.barplot(y=names,x=count)

In [17]:
group_list = []
for i in range(len(df)):
    if df.iloc[i].licensee_name in top5:
        groupID = top5.index(df.iloc[i].licensee_name)
    else:
        groupID = -1
    group_list.append(groupID)
df['group'] = group_list
df.groupby('group').size()

group
-1    174
 0    133
 1    111
 2     42
 3     10
 4      8
dtype: int64

In [29]:
df.to_csv('sg_supermarkets.csv')

# convert address to long lat

In [156]:
# this is an API , theres a limit on number of requests.

geolocator = Nominatim(user_agent="hello world!")
df['address'] =  df.block_house_num.astype(str)+" "+df.street_name.astype(str)

big_data = []

# for i in tqdm.tqdm(range(5)):
for i in tqdm.tqdm(range(df.shape[0])):
    data = {}
    data['id'] = i
    try:
        data['data'] = geolocator.geocode(df['address'][i]).raw
    except:
        data['id'] = i
        data['data'] = np.nan
    big_data.append(data)  
    
print(len(big_data))

with open ('geo_py_data.json','w') as f:
    json.dump(big_data,f)

100%|██████████| 478/478 [10:46<00:00,  1.34s/it]


478


In [206]:
big_data[:5]

[{'id': 0,
  'data': {'place_id': '57098306',
   'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
   'osm_type': 'node',
   'osm_id': '4631030455',
   'boundingbox': ['1.3808589', '1.3809589', '103.8791812', '103.8792812'],
   'lat': '1.3809089',
   'lon': '103.8792312',
   'display_name': 'Giant, 982, Buangkok Crescent, Buangkok, Northeast, 530982, Singapore',
   'class': 'shop',
   'type': 'supermarket',
   'importance': 0.31100000000000005,
   'icon': 'https://nominatim.openstreetmap.org/images/mapicons/shopping_supermarket.p.20.png'}},
 {'id': 1,
  'data': {'place_id': '60303373',
   'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
   'osm_type': 'node',
   'osm_id': '4869133717',
   'boundingbox': ['1.3792357', '1.3793357', '103.8878376', '103.8879376'],
   'lat': '1.3792857',
   'lon': '103.8878876',
   'display_name': 'UOB, 21, Hougang Street 51, Hougang, Northeast, 538719, Singapore',
   'class': 'amenity',

# load data for heatmap

In [170]:
df = pd.read_csv('sg_supermarkets.csv')

with open ('geo_py_data.json','r') as f:
    input_data = json.load(f)

df['geopyraw'] = input_data

In [183]:
def get_coords(cell):
    try:
        return (cell['data']['lat'],cell['data']['lon'])
    except:
        return np.nan
    
df['coords'] = df['geopyraw'].apply(get_coords)

In [188]:
df.coords[df.coords.isna()]

76     NaN
145    NaN
217    NaN
259    NaN
355    NaN
427    NaN
Name: coords, dtype: object

In [189]:
df.columns

Index(['Unnamed: 0', 'licence_num', 'licensee_name', 'building_name',
       'block_house_num', 'level_num', 'unit_num', 'street_name',
       'postal_code', 'group', 'address', 'geopyraw', 'coords'],
      dtype='object')

# show map

In [198]:
i=0
x,y = float(df.coords[i][0]),float(df.coords[i][1])
x
y

In [205]:
# define color map
cmap = {0:'red',
        1:'green',
        2:'blue',
        3:'yellow',
        4:'cyan',
        -1:'grey'}

# draw map
sg_start = (1.3521, 103.8198)
m = folium.Map(location=sg_start,tiles="OpenStreetMap")

error_c = 0
for i in range(0,len(df)-1):
# for i in range(0,50):
    try:
        x,y = float(df.coords[i][0]),float(df.coords[i][1])
        color = cmap[df.group[i]]
        folium.CircleMarker((x,y),
                            popup="",
                            tooltip="",
                            color=color,
                            fill_color=color,).add_to(m)
    except:
        error_c +=1
        pass

print(f"number of shops that cannot be plotted {error_c}")
    
m.save('map.html')
m

number of shops that cannot be plotted 6
